In [3]:
#adapted from write-inputs-cluster (Ivan)
import numpy as np

In [25]:
### Read j data ###
dataj_imp = np.loadtxt('jij.3layers.rot')

### Definitions ####
size_x = 80 # size of the spin lattice
size_y = 40
size_z = 3
va = [ 0.707000, 0.000000, 0] ## primitive vector 1 of the lattice
vb = [ 0.353500, 0.612300, 0] ## primitive vector 2 of the lattice
vc = [ 0.353500, 0.204100, 0.577300]
pos_imp = 1 # position (site number) of the impurity on jfile.imp

In [26]:
maxn = 0
for i in range(len(va)):
    # str(va[i])[::-1] invert the reading of the component i of vector va and .find('.') takes the position of the string
    # where the '.' is. In the end we find how many decimal places the number has.
    if int(str(va[i])[::-1].find('.')) >= maxn:
        maxn = int(str(va[i])[::-1].find('.'))
    # same logic but now for vb, we keep safe the maximal number of decimal places based on the 2 vectors
    if int(str(vb[i])[::-1].find('.')) >= maxn:
        maxn = int(str(vb[i])[::-1].find('.'))
    if int(str(vc[i])[::-1].find('.')) >= maxn:
        maxn = int(str(vc[i])[::-1].find('.'))
# the previous lines are useful for general primitve vectors

In [27]:
atom = 0 ; positions = [] ; center = 0
NAME=str(size_x)+'x'+str(size_y)


 #value between 0 and 1 to specify the position of the cluster inside the lattice
fx = 1/2  
fy = 1/2 

mean_x = round(size_x*fx)
mean_y = round(size_y*fy)
mean_z = 0
for k in range(size_z):
    for i in range(size_x):
        for j in range(size_y):
            p = (np.multiply(va,i)+np.multiply(vb,j)+np.multiply(vc,k))
            # for center use i=mean_x and j=mean_y
            positions.append(p.tolist())
            if i == mean_x and j == mean_y and k == mean_z:
                center = size_x*size_y*k+size_y*i+j+1 #index in lattice of pos_imp for the inclusion of cluster
positions = np.round(positions, decimals=maxn).tolist() #list of lattice position --> posfile will have ALL the lattice
print('[ok]')

[ok]


In [29]:
### write "pure" impurity interactions
#   and write posfile in tha same order firstly defined
print('writing file posfile_clus and separting cluster atoms index in lattice \n', end='')
posclus = open('posfile_clus_3layers_'+NAME, 'w')

interactions_center = list(np.where(dataj_imp[:,0]==pos_imp)[0])       #index in list of cluster atoms
dataj_imp_vec_center = dataj_imp[interactions_center,2:5]              #cluster atoms relative positions in respect to pos_imp

all_sites_imp = [] #index in lattice of cluster atoms with center, ordered as firstly defined

all_sites_imp.append(center)
print('%3s%3s' %(1, 1)  + '{0:12.6f} {1:12.6f} {2:12.6f}'.format(*positions[center-1]), file = posclus)

for i in range(len(dataj_imp_vec_center)):
    pos_now = np.add(positions[center-1],dataj_imp_vec_center[i]) #from center, finding position in lattice of cluster atoms
    pos_now = np.round(pos_now, decimals=maxn) 
    #print(i+2, pos_now)
    print('%3s%3s' %(i+2, i+2)  + '{0:12.6f} {1:12.6f} {2:12.6f}'.format(*pos_now), file = posclus)
    
    ind = int(positions.index(list(pos_now)))+1 #finding index in lattice from position
    all_sites_imp.append(ind)

#print(all_sites_imp)
posclus.close()
print('[ok]')
# posfile made keeping the order of index as in cluster definition

writing file posfile_clus and separting cluster atoms index in lattice 
2 [34.9965 11.6337  0.    ]
3 [36.764 12.246  0.   ]
4 [36.4105 11.6337  0.    ]
5 [36.057  11.0214  0.    ]
6 [36.4105 12.8583  0.    ]
7 [36.057 12.246  0.   ]
8 [35.7035 11.6337  0.    ]
9 [35.35   11.0214  0.    ]
10 [36.057  13.4706  0.    ]
11 [35.7035 12.8583  0.    ]
12 [34.643  11.0214  0.    ]
13 [35.35   13.4706  0.    ]
14 [34.9965 12.8583  0.    ]
15 [34.643 12.246  0.   ]
16 [34.2895 11.6337  0.    ]
17 [34.643  13.4706  0.    ]
18 [34.2895 12.8583  0.    ]
19 [33.936 12.246  0.   ]
20 [34.9965 12.4501  0.5773]
21 [36.4105 12.4501  0.5773]
22 [36.057  11.8378  0.5773]
23 [35.7035 11.2255  0.5773]
24 [36.057  13.0624  0.5773]
25 [35.7035 12.4501  0.5773]
26 [35.35   11.8378  0.5773]
27 [34.9965 11.2255  0.5773]
28 [35.7035 13.6747  0.5773]
29 [35.35   13.0624  0.5773]
30 [34.643  11.8378  0.5773]
31 [34.2895 11.2255  0.5773]
32 [34.9965 13.6747  0.5773]
33 [34.643  13.0624  0.5773]
34 [34.2895 12.4501 

In [35]:
### write a *.xyz file with 'Fe' format as the impurities and 'Nb' as the pristine
#   index order of atoms cluster respect lattice index order
print('writing file .xyz \n', end='')
system_file = open('system_3layers_'+NAME+'.xyz', 'w')

print(str(len(positions)) + '\n', file = system_file)
count = 1
for i in range(len(positions)):
    pos_now = positions[i]
    pos_now = np.round(pos_now, decimals=maxn)
    if i+1 in all_sites_imp:
        print("Nb " + '{0:12.6f} {1:12.6f} {2:12.6f}'.format(*pos_now), file = system_file)
        print(pos_now, i+1)
        count += 1
    else:
        if i<3200:
            print("Ir" + '{0:12.6f} {1:12.6f} {2:12.6f}'.format(*pos_now), file = system_file)
        if i>6400:
            print("Pd" + '{0:12.6f} {1:12.6f} {2:12.6f}'.format(*pos_now), file = system_file)
        if i>=3200 and i<=6400:
            print("Fe" + '{0:12.6f} {1:12.6f} {2:12.6f}'.format(*pos_now), file = system_file)

system_file.close()

print('[ok]')

writing file .xyz 
[33.936 12.246  0.   ] 1541
[34.2895 12.8583  0.    ] 1542
[34.643  13.4706  0.    ] 1543
[34.2895 11.6337  0.    ] 1580
[34.643 12.246  0.   ] 1581
[34.9965 12.8583  0.    ] 1582
[35.35   13.4706  0.    ] 1583
[34.643  11.0214  0.    ] 1619
[34.9965 11.6337  0.    ] 1620
[35.35  12.246  0.   ] 1621
[35.7035 12.8583  0.    ] 1622
[36.057  13.4706  0.    ] 1623
[35.35   11.0214  0.    ] 1659
[35.7035 11.6337  0.    ] 1660
[36.057 12.246  0.   ] 1661
[36.4105 12.8583  0.    ] 1662
[36.057  11.0214  0.    ] 1699
[36.4105 11.6337  0.    ] 1700
[36.764 12.246  0.   ] 1701
[33.5825 12.4501  0.5773] 4701
[33.936  13.0624  0.5773] 4702
[34.2895 13.6747  0.5773] 4703
[33.936  11.8378  0.5773] 4740
[34.2895 12.4501  0.5773] 4741
[34.643  13.0624  0.5773] 4742
[34.9965 13.6747  0.5773] 4743
[34.2895 11.2255  0.5773] 4779
[34.643  11.8378  0.5773] 4780
[34.9965 12.4501  0.5773] 4781
[35.35   13.0624  0.5773] 4782
[35.7035 13.6747  0.5773] 4783
[34.9965 11.2255  0.5773] 4819
[35.

In [31]:
# jmol file for psofile_clus, index order the same as cluster firstly defined
posclus = open('posfile_clus_3layers_'+NAME, 'r')
linhas = posclus.readlines()

pos_clus_xyz = open('posfile_clus_3layers.xyz', 'w')

print(str(len(linhas)) + '\n', file = pos_clus_xyz)

meio = linhas[0].split()
meio = meio[2:5]
print(meio)

for i in range(0,len(linhas)):
    diff = list(np.empty(3))
    outro = linhas[i].split()
    outro = outro[2:5]
    for j in range(3):
        diff[j] = float(outro[j]) - float(meio[j])
    print(diff)
    print('Fe' + '{0:12.6f} {1:12.6f} {2:12.6f}'.format(*diff), file = pos_clus_xyz)

pos_clus_xyz.close()
posclus.close()

['35.350000', '12.246000', '0.000000']
[0.0, 0.0, 0.0]
[-0.3535000000000039, -0.6123000000000012, 0.0]
[1.4140000000000015, 0.0, 0.0]
[1.0604999999999976, -0.6123000000000012, 0.0]
[0.7070000000000007, -1.2246000000000006, 0.0]
[1.0604999999999976, 0.6122999999999994, 0.0]
[0.7070000000000007, 0.0, 0.0]
[0.3534999999999968, -0.6123000000000012, 0.0]
[0.0, -1.2246000000000006, 0.0]
[0.7070000000000007, 1.2245999999999988, 0.0]
[0.3534999999999968, 0.6122999999999994, 0.0]
[-0.7070000000000007, -1.2246000000000006, 0.0]
[0.0, 1.2245999999999988, 0.0]
[-0.3535000000000039, 0.6122999999999994, 0.0]
[-0.7070000000000007, 0.0, 0.0]
[-1.0605000000000047, -0.6123000000000012, 0.0]
[-0.7070000000000007, 1.2245999999999988, 0.0]
[-1.0605000000000047, 0.6122999999999994, 0.0]
[-1.4140000000000015, 0.0, 0.0]
[-0.3535000000000039, 0.2041000000000004, 0.5773]
[1.0604999999999976, 0.2041000000000004, 0.5773]
[0.7070000000000007, -0.4082000000000008, 0.5773]
[0.3534999999999968, -1.0205000000000002, 0